In [1]:
import chromadb
chroma_client = chromadb.Client()

In [3]:
collection = chroma_client.create_collection(name="iqs")

In [4]:
collection.add(
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

C:\Users\rosha\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [02:33<00:00, 542kiB/s]   


In [12]:
print(collection.query(
    query_texts=["When did India get Independence"],
    n_results=2
))

{'ids': [['id3', 'id2']], 'distances': [[0.254454106092453, 2.057119846343994]], 'metadatas': [[None, {'source': 'my_source'}]], 'embeddings': None, 'documents': [['India got independence on 1947', 'This is another document']], 'uris': None, 'data': None}


In [10]:
collection.add(documents = ['India got independence on 1947'], ids = ['id3'])

In [13]:
from langchain_community.document_loaders import TextLoader


In [15]:
raw_documents = TextLoader('test.txt').load()


In [16]:
print(raw_documents)

[Document(page_content='blah blah', metadata={'source': 'test.txt'})]


In [26]:
collection = client.get_collection(name="iqs")

In [24]:
import chromadb

client = chromadb.Client()


In [29]:
print(collection.to_dict())

AttributeError: 'Collection' object has no attribute 'to_dict'